In [ ]:
import numpy as np
import pandas as pd
import random
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
      print(os.path.join(dirname, filename))
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
import matplotlib.pyplot as plt
import string
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import timeit

filepath_train= ('/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
filepath_test = ('/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

def import_data(filepath):
    X = pd.read_csv(filepath).drop('label', axis=1)
    y = pd.read_csv(filepath).loc[:, 'label']

    return (np.array(X), np.array(y))

X_train, y_train = import_data(filepath_train)
X_test, y_test = import_data(filepath_test)


for i in range(10, 25, 1):
    y_train[y_train == i] = i - 1
    y_test[y_test == i] = i - 1
print('Shape of X_train : {} , Shape of X_test : {}'.format(X_train.shape, X_test.shape))
print('Shape of y_train : {}  Shape of y_test : {}'.format(y_train.shape, y_test.shape))



num_letters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F',
               6: 'G', 7: 'H', 8: 'I', 9: 'K',
               10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P',
               15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U',
               20: 'V', 21: 'W', 22: 'X', 23: 'Y'}

plt.figure(figsize=(10, 10))

for i in range(9):
    ind = random.randint(0, X_train.shape[0])

    plt.subplot(3,3, i + 1)
    plt.imshow(X_train[ind].reshape(28, 28), cmap='gray')

    plt.title(num_letters[y_train[ind]], fontdict={'color': 'Black', 'fontsize': 20})

plt.show()


X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

plt.figure(figsize = (30,10))

plt.xticks(size=10)
sns.countplot(x = y_train)
plt.title('Frequency of each label')
plt.show()

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 10,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  shear_range=0,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

X_test=X_test/255


y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

model=Sequential()
model.add(Conv2D(128,kernel_size=(5,5),
                 strides=1,padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=2,padding='same'))
model.add(Conv2D(64,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
model.add(Conv2D(32,kernel_size=(2,2),
                strides=1,activation='relu',padding='same'))
model.add(MaxPool2D((2,2),2,padding='same'))
          
model.add(Flatten())

model.add(Dense(units=512,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=24,activation='softmax'))
model.summary()
# model.compile(optimizer='adam',metrics = ['accuracy'],loss = 'categorical_crossentropy')
# model.fit(train_datagen.flow(X_train,y_train,batch_size = 300),validation_data=(X_test,y_test),epochs = 50)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_datagen.flow(X_train,y_train,batch_size=400), epochs = 50,validation_data=(X_test,y_test),shuffle=1)

y_pred = model.predict(X_test)
y_pred2 = np.array([np.argmax(y_pred[i]) for i in range(len(y_pred))])
y_test2 = np.array([np.argmax(y_test[i]) for i in range(len(y_test))])
# plt.figure(figsize = (15,15))

# sns.heatmap(confusion_matrix(y_test2,y_pred2),annot= True,cbar = False , cmap = 'Blues',annot_kws={'size' : 8})
# plt.xlabel('Predicted',fontdict={'size': 20})
# plt.ylabel('True' ,fontdict={'size': 20})
# plt.title('Confusion Matrix',fontdict={'size': 20,'color':'Blue'})
# plt.show()
plt.figure(figsize = (15,15))
for i in range(10):
  ind = np.random.randint(0,len(X_test))
  plt.subplot(5,2,i+1)
  plt.imshow(X_test[ind].reshape(28,28),cmap = 'gray')
  plt.title('True : {} , Predicted : {} with {} % of probabilities '.format(num_letters[y_test2[ind]],
                                                                            num_letters[y_pred2[ind]],
                                                                            round(y_pred[ind][y_pred2[ind]]*100,2)
                                                                            ))
  plt.show()
(ls,acc)=model.evaluate(x=X_test,y=y_test)
print('MODEL ACCURACY = {}%'.format(acc*100))